In [1]:
library(caret, quiet=TRUE);
library(base64enc)
library(httr, quiet=TRUE)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:
set.seed(1960)

create_model  =  function() {

    model  <- train(Species ~ ., data = iris, method = "rpart" , preProcess = c("invHyperbolicSine"))
    
    return(model)
}

In [3]:
# dataset
model = create_model()

In [4]:
pred <- predict(model, as.matrix(iris[, -5]) , type="prob")
pred_labels <- predict(model, as.matrix(iris[, -5]) , type="raw")
sum(pred_labels != iris$Species)/length(pred_labels)

[1] 0.04

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "caret_rpart_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)

In [7]:
cat(lModelSQL)


WITH "ADS_pre_1_OUT" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_0" + sqrt("ADS"."Feature_0" * "ADS"."Feature_0" + 1.0) IS NULL OR "ADS"."Feature_0" + sqrt("ADS"."Feature_0" * "ADS"."Feature_0" + 1.0) > 0.0) THEN ln("ADS"."Feature_0" + sqrt("ADS"."Feature_0" * "ADS"."Feature_0" + 1.0)) ELSE -1.79769313486231e+308 END AS "Petal.Length", CASE WHEN ("ADS"."Feature_1" + sqrt("ADS"."Feature_1" * "ADS"."Feature_1" + 1.0) IS NULL OR "ADS"."Feature_1" + sqrt("ADS"."Feature_1" * "ADS"."Feature_1" + 1.0) > 0.0) THEN ln("ADS"."Feature_1" + sqrt("ADS"."Feature_1" * "ADS"."Feature_1" + 1.0)) ELSE -1.79769313486231e+308 END AS "Petal.Width", CASE WHEN ("ADS"."Feature_2" + sqrt("ADS"."Feature_2" * "ADS"."Feature_2" + 1.0) IS NULL OR "ADS"."Feature_2" + sqrt("ADS"."Feature_2" * "ADS"."Feature_2" + 1.0) > 0.0) THEN ln("ADS"."Feature_2" + sqrt("ADS"."Feature_2" * "ADS"."Feature_2" + 1.0)) ELSE -1.79769313486231e+308 END AS "Sepal.Length", CASE WHEN ("ADS"."Feature_3" + sqrt("ADS"."Featur

# Execute the SQL Code

In [8]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [9]:
dataset = iris[,-5]

df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

Feature_0,Feature_1,Feature_2,Feature_3,KEY
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,2
4.7,3.2,1.3,0.2,3
4.6,3.1,1.5,0.2,4
5.0,3.6,1.4,0.2,5
5.4,3.9,1.7,0.4,6


In [10]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [11]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

KEY,Score_setosa,Score_versicolor,Score_virginica,Proba_setosa,Proba_versicolor,Proba_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
1,NA,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
2,NA,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
3,NA,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
4,NA,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
5,NA,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
6,NA,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609


# R Caret Rpart Output

In [12]:
pred_proba  =  predict(model, as.matrix(iris[,-5]), type = "prob")
df_r_out = data.frame(pred_proba)
names(df_r_out) = sprintf("Proba_%s",model$levels)

df_r_out$KEY = seq.int(nrow(dataset))
df_r_out$Score_setosa  =  NA
df_r_out$Score_versicolor  =  NA
df_r_out$Score_virginica  =  NA
df_r_out$LogProba_setosa  =  log(df_r_out$Proba_setosa)
df_r_out$LogProba_versicolor =  log(df_r_out$Proba_versicolor)
df_r_out$LogProba_virginica  =  log(df_r_out$Proba_virginica)
df_r_out$Decision =   predict(model, as.matrix(iris[,-5]), type = "raw")
df_r_out$DecisionProba =  apply(pred_proba, 1, function(x) max(x))
head(df_r_out)

Proba_setosa,Proba_versicolor,Proba_virginica,KEY,Score_setosa,Score_versicolor,Score_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
1,0,0,1,NA,NA,NA,0,-Inf,-Inf,setosa,1
1,0,0,2,NA,NA,NA,0,-Inf,-Inf,setosa,1
1,0,0,3,NA,NA,NA,0,-Inf,-Inf,setosa,1
1,0,0,4,NA,NA,NA,0,-Inf,-Inf,setosa,1
1,0,0,5,NA,NA,NA,0,-Inf,-Inf,setosa,1
1,0,0,6,NA,NA,NA,0,-Inf,-Inf,setosa,1


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_R","_SQL"))
head(df_merge)

KEY,Proba_setosa_R,Proba_versicolor_R,Proba_virginica_R,Score_setosa_R,Score_versicolor_R,Score_virginica_R,LogProba_setosa_R,LogProba_versicolor_R,LogProba_virginica_R,⋯,Score_versicolor_SQL,Score_virginica_SQL,Proba_setosa_SQL,Proba_versicolor_SQL,Proba_virginica_SQL,LogProba_setosa_SQL,LogProba_versicolor_SQL,LogProba_virginica_SQL,Decision_SQL,DecisionProba_SQL
1,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
2,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
3,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
4,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
5,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609
6,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,0,0.02173913,0.9782609,-1.797693e+307,-3.828641,-0.02197891,virginica,0.9782609


In [14]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”

KEY,Proba_setosa_R,Proba_versicolor_R,Proba_virginica_R,Score_setosa_R,Score_versicolor_R,Score_virginica_R,LogProba_setosa_R,LogProba_versicolor_R,LogProba_virginica_R,⋯,Score_versicolor_SQL,Score_virginica_SQL,Proba_setosa_SQL,Proba_versicolor_SQL,Proba_virginica_SQL,LogProba_setosa_SQL,LogProba_versicolor_SQL,LogProba_virginica_SQL,Decision_SQL,DecisionProba_SQL


In [15]:
stopifnot(nrow(diffs_df) == 0)

In [16]:
summary(df_sql_out)

      KEY         Score_setosa   Score_versicolor Score_virginica  Proba_setosa
 Min.   :  1.00   Mode:logical   Mode:logical     Mode:logical    Min.   :0    
 1st Qu.: 38.25   NA's:150       NA's:150         NA's:150        1st Qu.:0    
 Median : 75.50                                                   Median :0    
 Mean   : 75.50                                                   Mean   :0    
 3rd Qu.:112.75                                                   3rd Qu.:0    
 Max.   :150.00                                                   Max.   :0    
 Proba_versicolor  Proba_virginica  LogProba_setosa       LogProba_versicolor
 Min.   :0.02174   Min.   :0.9783   Min.   :-1.798e+307   Min.   :-3.829     
 1st Qu.:0.02174   1st Qu.:0.9783   1st Qu.:-1.798e+307   1st Qu.:-3.829     
 Median :0.02174   Median :0.9783   Median :-1.798e+307   Median :-3.829     
 Mean   :0.02174   Mean   :0.9783   Mean   :-1.798e+307   Mean   :-3.829     
 3rd Qu.:0.02174   3rd Qu.:0.9783   3rd Qu.:-1.798

In [17]:
summary(df_r_out)

  Proba_setosa    Proba_versicolor  Proba_virginica        KEY        
 Min.   :0.0000   Min.   :0.00000   Min.   :0.00000   Min.   :  1.00  
 1st Qu.:0.0000   1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.: 38.25  
 Median :0.0000   Median :0.02174   Median :0.09259   Median : 75.50  
 Mean   :0.3333   Mean   :0.33333   Mean   :0.33333   Mean   : 75.50  
 3rd Qu.:1.0000   3rd Qu.:0.90741   3rd Qu.:0.97826   3rd Qu.:112.75  
 Max.   :1.0000   Max.   :0.90741   Max.   :0.97826   Max.   :150.00  
 Score_setosa   Score_versicolor Score_virginica LogProba_setosa
 Mode:logical   Mode:logical     Mode:logical    Min.   :-Inf   
 NA's:150       NA's:150         NA's:150        1st Qu.:-Inf   
                                                 Median :-Inf   
                                                 Mean   :-Inf   
                                                 3rd Qu.:   0   
                                                 Max.   :   0   
 LogProba_versicolor LogProba_virginica       De

In [18]:
prep = model$preProcess

In [19]:
prep

Created from 150 samples and 4 variables

Pre-processing:
  - ignored (0)
  - Inverve Hyperbolic Sine transformation (4)
